# Построение воходных конвейеров с использованием tf.data - API-интерфейса Dataset библиотеки

Когда набор данны слишком большой и потому не умущается в памяти, нам понадобится загружать данные из основного устройства хранения порциями, т.е. пакет за пакетом.

## Создание объекта Dataset из существующих тензоров

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np
np.set_printoptions(precision=3)

In [4]:
a = [1.2, 3.4, 7.5, 4.1, 5.0, 1.0]
ds = tf.data.Dataset.from_tensor_slices(a)
print(ds)

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.float32, name=None)>


Функция возвратила объект класса `Dataset`, который можно применить для прохода по индивидуальным элементам во входном наборе данных.

In [5]:
for item in ds:
    print(item)

tf.Tensor(1.2, shape=(), dtype=float32)
tf.Tensor(3.4, shape=(), dtype=float32)
tf.Tensor(7.5, shape=(), dtype=float32)
tf.Tensor(4.1, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


Создатим из этого набора данных пакеты с размером 2:

In [8]:
ds_batch = ds.batch(2)
for i, elem in enumerate(ds_batch, 1):
    print('пакет {}:' .format(i), elem.numpy())

пакет 1: [1.2 3.4]
пакет 2: [7.5 4.1]
пакет 3: [5. 1.]


Если количество элементов в тензоре не желаемому зармеру пакета (batch):

In [9]:
ds_batch = ds.batch(4, drop_remainder=False)
for i, elem in enumerate(ds_batch, 1):
    print('пакет {}:' .format(i), elem.numpy())

пакет 1: [1.2 3.4 7.5 4.1]
пакет 2: [5. 1.]


In [10]:
ds_batch = ds.batch(4, drop_remainder=True)
for i, elem in enumerate(ds_batch, 1):
    print('пакет {}:' .format(i), elem.numpy())

пакет 1: [1.2 3.4 7.5 4.1]


## Объединение двух тензоров в общий набор данных

Часто данные находятся в двух или большем числе тензоров, напимер, тензор признаков и тензор меток. В таких случаях надо построить набор данных, который объединит эти тензоры и позволит извлекать элементы тензоров в кортежах.

In [13]:
tf.random.set_seed(1)
t_x = tf.random.uniform([4, 3], dtype=tf.float32)
t_y = tf.range(4)

print(t_x.numpy())
print(t_y.numpy())

[[0.165 0.901 0.631]
 [0.435 0.292 0.643]
 [0.976 0.435 0.66 ]
 [0.605 0.637 0.614]]
[0 1 2 3]


Создадим общий набор данных:
1. Создадим два набора данных
1. Построим общий набор данных, используя фукнцию `.zip()`

In [17]:
ds_x = tf.data.Dataset.from_tensor_slices(t_x)
ds_y = tf.data.Dataset.from_tensor_slices(t_y)
# Объединим объекты Dataset в один:
ds_joint = tf.data.Dataset.zip((ds_x, ds_y))

Тензоры объединеты в кортежи:

In [18]:
for example in ds_joint:
    print(example)

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.165, 0.901, 0.631], dtype=float32)>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.435, 0.292, 0.643], dtype=float32)>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.976, 0.435, 0.66 ], dtype=float32)>, <tf.Tensor: shape=(), dtype=int32, numpy=2>)
(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.605, 0.637, 0.614], dtype=float32)>, <tf.Tensor: shape=(), dtype=int32, numpy=3>)


Поличим данные из кортежей:

In [19]:
for example in ds_joint:
    print('x:', example[0].numpy(),
          'y:', example[1].numpy())

x: [0.165 0.901 0.631] y: 0
x: [0.435 0.292 0.643] y: 1
x: [0.976 0.435 0.66 ] y: 2
x: [0.605 0.637 0.614] y: 3


Все это можно сделать, используя метод `tf.data.Dataset.from_tensor_slices():

In [20]:
ds_jount = tf.data.Dataset.from_tensor_slices((t_x, t_y))
for example in ds_joint:
    print('x:', example[0].numpy(),
          'y:', example[1].numpy())

x: [0.165 0.901 0.631] y: 0
x: [0.435 0.292 0.643] y: 1
x: [0.976 0.435 0.66 ] y: 2
x: [0.605 0.637 0.614] y: 3


## Применение трансформации к отдельным элементам набора данных

In [38]:
# Трансформируем набор признаков:
ds_trans = ds_joint.map(lambda x, y: (x*2-1, y))
for example in ds_trans:
    print(' x:', example[0].numpy(),
          ' y:', example[1].numpy())

 x: [-0.67   0.803  0.262]  y: 0
 x: [-0.131 -0.416  0.285]  y: 1
 x: [ 0.952 -0.13   0.32 ]  y: 2
 x: [0.21  0.273 0.229]  y: 3
